In [3]:
import numpy as np
import pandas as pd
import h5py
import pysam

In [4]:
fasta_ref = pysam.FastaFile('../reference/hg38.genome.fa')

In [5]:
sequences = []

In [7]:
peaks_df = pd.read_csv('../ENCSR000EGM/data/origp.bed', sep='\t', header=None, names=['chrom', 'st', 'end', 'name', 'score', 'strand', 'signalValue', 'p', 'q', 'summit'])

In [9]:
peaks_df = peaks_df[peaks_df['chrom'].isin(('chr1',))]

In [10]:
peaks_df.head()

,chrom,st,end,name,score,strand,signalValue,p,q,summit
3,chr1,35641559,35641799,.,1000,.,6.52180,-1.0,0.00080,120
6,chr1,27600557,27600797,.,599,.,6.59866,-1.0,0.00209,120
10,chr1,1944232,1944472,.,1000,.,6.69948,-1.0,0.01882,120
17,chr1,159925706,159925946,.,598,.,6.81884,-1.0,0.03458,120
24,chr1,7954373,7954613,.,579,.,6.88965,-1.0,0.04601,120


In [11]:
peaks_df = peaks_df.reset_index(drop=True)

In [12]:
peaks_df.head()

,chrom,st,end,name,score,strand,signalValue,p,q,summit
0,chr1,35641559,35641799,.,1000,.,6.52180,-1.0,0.00080,120
1,chr1,27600557,27600797,.,599,.,6.59866,-1.0,0.00209,120
2,chr1,1944232,1944472,.,1000,.,6.69948,-1.0,0.01882,120
3,chr1,159925706,159925946,.,598,.,6.81884,-1.0,0.03458,120
4,chr1,7954373,7954613,.,579,.,6.88965,-1.0,0.04601,120


In [13]:
peaks_df['start'] = peaks_df['st'] + peaks_df['summit'] - \
    (2114 // 2)
peaks_df['end'] = peaks_df['st'] + peaks_df['summit'] + \
    (2114 // 2)

In [34]:
peaks_df.head()

,chrom,st,end,name,score,strand,signalValue,p,q,summit,start
0,chr1,35641559,35642736,.,1000,.,6.52180,-1.0,0.00080,120,35640622
1,chr1,27600557,27601734,.,599,.,6.59866,-1.0,0.00209,120,27599620
2,chr1,1944232,1945409,.,1000,.,6.69948,-1.0,0.01882,120,1943295
3,chr1,159925706,159926883,.,598,.,6.81884,-1.0,0.03458,120,159924769
4,chr1,7954373,7955550,.,579,.,6.88965,-1.0,0.04601,120,7953436


In [15]:
num_peaks = peaks_df.shape[0]
print(num_peaks)

6085


In [42]:
seq1 = fasta_ref.fetch('chr1', 35640622, 35642736).upper()
print(seq1, type(seq1), len(seq1))

AGCGACACCAGGAAACTTATCAGAGATTCGGGGATCTAGAGAGCCAGTCCCACAGTCTCACTCTCCTCAGGATCCCACCTTATGCCGCCAACAAACTAACCTCATTACTAGCGACACCAGGAAACTTATCAGAGATTCGGGGATCTAGAGAGCCAGGCTCTGGGCAGGCGCTTAAACATTTATAAACTGATTCCCTTTAGGCAACTCTCACCAAGCAAGAAGCATTTAGCGAAGGCCAGAGCAGAGCACTGACCCAGGAGCCCTGGAGGAGCAAATGCCAAGGTCCCAGACCTGTCTGCCTGTTTCGCCTGTAATCCCAGCACTTTGGAAGGCCGAGACGGGCGGATCTCCTGAGGTCGGGAGTTCGAGACCAGCCTGACCAATATGGAGAAATCCACATCTCTACTAATAATACAAAAATTAGCCAGGCGTGGTGGCGCATATCTGTTCCCAGCTACTCGGGAGGCTGAGGCACGAGAATCGCTTGAACCCGGGAGACAAAGACTGTGCTAAGCCGAGATCGCGCCATTGCACTCCGGCCTGGACAACAAGAGCGAAACCCCTCTCGAAAAAATAAATAAATAAATAAAATAAGTACGGGCAGGGCAGGCCGGCTTCCATCTCTCAGATCCTCCCTGGTACTTATTCAACCCCCGACAAACTCCTTCTGGCCGCTCCTACACCCCAGGCCTGGCCGGGCCTCCCCTGCTTCCTCTCACCGTCCTTCATCTGGACAATATTGCTGGCGGCCACCCGGTCGGAGGCGACAAGAACATAGTCGGGGCCTTGGATACCGATGAGGTACTCCATGGTGGCGGAAGGCCAGGGGCTGCAGGTCCGACACAGCACGAGACTCGCCCGCTTCCAGGTCTCACCGGTGAGACAGCACCTCAGAGCGAAGATTGGCGCGACGCCTGCAGCACGACTTCCACGGCGCTCTCGGATGACGTACAACTGTCGCGAGAGGTTGCAAAGCGGGCGCGGCGCCGGGTGCCTTATG

In [44]:
def insert_variant(seq, allele, position):
    left, right = seq[:position-1], seq[position:]
    return left + allele + right

In [43]:
seq1_var = insert_variant(seq1, 'G', 1057)
print(seq1_var, len(seq1_var))

AGCGACACCAGGAAACTTATCAGAGATTCGGGGATCTAGAGAGCCAGTCCCACAGTCTCACTCTCCTCAGGATCCCACCTTATGCCGCCAACAAACTAACCTCATTACTAGCGACACCAGGAAACTTATCAGAGATTCGGGGATCTAGAGAGCCAGGCTCTGGGCAGGCGCTTAAACATTTATAAACTGATTCCCTTTAGGCAACTCTCACCAAGCAAGAAGCATTTAGCGAAGGCCAGAGCAGAGCACTGACCCAGGAGCCCTGGAGGAGCAAATGCCAAGGTCCCAGACCTGTCTGCCTGTTTCGCCTGTAATCCCAGCACTTTGGAAGGCCGAGACGGGCGGATCTCCTGAGGTCGGGAGTTCGAGACCAGCCTGACCAATATGGAGAAATCCACATCTCTACTAATAATACAAAAATTAGCCAGGCGTGGTGGCGCATATCTGTTCCCAGCTACTCGGGAGGCTGAGGCACGAGAATCGCTTGAACCCGGGAGACAAAGACTGTGCTAAGCCGAGATCGCGCCATTGCACTCCGGCCTGGACAACAAGAGCGAAACCCCTCTCGAAAAAATAAATAAATAAATAAAATAAGTACGGGCAGGGCAGGCCGGCTTCCATCTCTCAGATCCTCCCTGGTACTTATTCAACCCCCGACAAACTCCTTCTGGCCGCTCCTACACCCCAGGCCTGGCCGGGCCTCCCCTGCTTCCTCTCACCGTCCTTCATCTGGACAATATTGCTGGCGGCCACCCGGTCGGAGGCGACAAGAACATAGTCGGGGCCTTGGATACCGATGAGGTACTCCATGGTGGCGGAAGGCCAGGGGCTGCAGGTCCGACACAGCACGAGACTCGCCCGCTTCCAGGTCTCACCGGTGAGACAGCACCTCAGAGCGAAGATTGGCGCGACGCCTGCAGCACGACTTCCACGGCGCTCTCGGATGACGTACAACTGTCGCGAGAGGTTGCAAAGCGGGCGCGGCGCCGGGTGCCTTATG

In [48]:
def make_variant_input(chromosome, position, allele1, allele2):
    entry = [[chromosome, position, allele1, allele2]]
    df = pd.DataFrame(entry, columns = \
        ['chrom', 'position', 'allele1', 'allele2'])
    return df

In [52]:
var_df = make_variant_input('chr1', 35641679, 'A', 'G')
var_df.info()
var_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   chrom     1 non-null      object
 1   position  1 non-null      int64 
 2   allele1   1 non-null      object
 3   allele2   1 non-null      object
dtypes: int64(1), object(3)
memory usage: 160.0+ bytes


,chrom,position,allele1,allele2
0,chr1,35641679,A,G
